In [1]:
import tensorflow as tf
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from numpy.random import RandomState

In [142]:
class Actor(tf.keras.Model):
    def __init__(self, in_features=100, out_features=18):
        super(Actor, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        self.linear_1 = tf.keras.layers.Dense(in_features, activation='relu')
        self.linear_2 = tf.keras.layers.Dense(in_features, activation='relu')
        self.linear_3 = tf.keras.layers.Dense(out_features, activation='tanh')
        
    def call(self, state):
        inputs = tf.reshape(state, (1, self.in_features))
        print(tf.shape(inputs))
        output = self.linear_1(inputs)
        output = self.linear_2(output)
        output = self.linear_3(output)
        return output

In [143]:
## instansiasi model
actorNets = Actor(100, 18)

In [134]:
random_state = RandomState(1)

state = np.random.rand(100,)
state

array([0.63345062, 0.27934384, 0.47266281, 0.91321615, 0.44804094,
       0.23915187, 0.62378167, 0.73143038, 0.58242637, 0.61717857,
       0.26533823, 0.2068301 , 0.72555059, 0.09991472, 0.14279929,
       0.14738349, 0.65841297, 0.65331981, 0.8732705 , 0.06552706,
       0.54172127, 0.17189618, 0.98920991, 0.2121848 , 0.90012836,
       0.13123975, 0.39300696, 0.72673111, 0.18174008, 0.78040402,
       0.56635225, 0.21580389, 0.39304532, 0.81361001, 0.65960383,
       0.18184047, 0.42159888, 0.06221726, 0.33183348, 0.31202788,
       0.47642669, 0.64520161, 0.53443925, 0.15941396, 0.87408018,
       0.37055042, 0.58742235, 0.22557801, 0.67680067, 0.83373564,
       0.09652874, 0.19499143, 0.45669526, 0.26955529, 0.25067314,
       0.55347439, 0.83942066, 0.69380528, 0.81496299, 0.77192226,
       0.81368896, 0.71146776, 0.61279881, 0.20697145, 0.91646921,
       0.6149896 , 0.67317277, 0.16705778, 0.06535899, 0.22088057,
       0.52667785, 0.12176059, 0.46347153, 0.68557767, 0.88793

In [144]:
## output has two dimension so need to be flattened
tf.size(actorNets(state))

tf.Tensor([  1 100], shape=(2,), dtype=int32)


<tf.Tensor: shape=(), dtype=int32, numpy=18>

In [137]:
actorNets.summary()

Model: "actor_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            multiple                  10100     
                                                                 
 dense_49 (Dense)            multiple                  10100     
                                                                 
 dense_50 (Dense)            multiple                  1818      
                                                                 
Total params: 22,018
Trainable params: 22,018
Non-trainable params: 0
_________________________________________________________________


In [146]:
class Critic(tf.keras.Model):
    def __init__(self, action_size=20, in_features=128, out_features=18):
        super(Critic, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.combo_features = in_features + action_size
        self.action_size = action_size
        ## check shape of the input
        self.linear_1 = tf.keras.layers.Dense(self.in_features, 
                                              activation='relu')
        
        self.linear_2 = tf.keras.layers.Dense(self.combo_features, 
                                              activation='relu')
        
        self.linear_3 = tf.keras.layers.Dense(self.combo_features, 
                                              activation='relu')
        
        self.linear_4 = tf.keras.layers.Dense(out_features, 
                                              activation=None)
        
    def call(self, state, action):
        outputs = self.linear_1(state)
        outputs = self.linear_2(tf.concat([action, output]))
        outputs = self.linear_3(outputs)
        outputs = self.linear_4(outputs)
        return outputs